# Intro-Aufgabe für das SEP Unsicherheit im Maschinellen Lernen WS 2025/26

Mit diesem Notebook sollen Sie sich mit der Arbeit an probly vertraut machen.
Bevor Sie loslegen können, führen Sie bitte die folgenden Schritte durch:

1. Legen Sie einen GitHub-Account an und hinterlegen Sie in diesem einen SSH-Key für Ihren Laptop/PC ([Guide](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account)).
1. Klonen Sie das probly-Repo: `git clone git@github.com:pwhofman/probly.git`.
2. Installieren Sie das Tool uv ([Guide](https://docs.astral.sh/uv/getting-started/installation/)).
3. Navigieren Sie in Ihr lokales probly-Repo und installieren Sie die Dependencies: `uv sync`
4. Kopieren Sie dieses Notebook in Ihr lokales probly-Repo und führen Sie dort anschließend die folgende Code-Zelle aus:

In [1]:
import pytraverse as t

t.__doc__

'Generic functional datastructure traverser utilities.'

Wenn Sie die Ausgabe `Generic functional datastructure traverser utilities.` erhalten haben, haben Sie das Repository korrekt aufgesetzt.

## Was ist PyTraverse?

PyTraverse bildet den Kern der Modelltransformationsfeatures von probly.
In diesem Notebook sollen Sie sich daher damit vertraut machen.
Lesen Sie sich hierfür zuerst das [PyTraverse-Tutorial](https://github.com/pwhofman/probly/blob/main/notebooks/examples/pytraverse_tutorial.ipynb) in `notebooks/examples/pytraverse_tutorial.ipynb` an.

Nachdem Sie das Tutorial-Notebook gelesen (und verstanden) haben, können Sie sich an Ihren ersten ersten eigenen Traverser wagen.

## Der Verdopplungs-Traverser

Um Ihr Verständnis auf die Probe zu stellen, sollen Sie nun einen *Verdopplungs-Traverser* implementieren.
Dieser Traverser soll rekrusiv durch beliebig geschachtelte `list`, `tuple`, `set` und `dict` Datenstrukturen traversieren und dabei jeweils folgende Änderungen an den besuchtne Datenstrukturen durchführen:

1. `str`: Strings sollen mit sich selbt konkateniert werden, d.h., aus "Wau" wird "WauWau".
2. `int` und `float`: Gleitkomma- und Ganzzahlen sollen mit `2` multipliziert werden.
3. `list` und `tuple`: Auch Listen und Tupel sollen verdoppelt werden, d.h., aus `["a", 3]` wird `["aa", 6, "aa", 6]` (für Tupel analog).
4. `set` und `dict`: Mengen und Dicts lassen sich nicht sinnvoll verdoppeln; hier soll die Verdopplung also einfach nur auf alle Elemente angewandt werden (bei `dict`s sollen Key **und** Value verdoppelt werden), d.h. aus `{"x": 1, 3: ["a"]}` wird `{"xx": 2, 6: ["aa","aa"]}`.

Alle anderen Datentypen sollen unverändert bleiben.

Laden Sie dieses Notebook mit dem implementierten Traverser auf Moodle hoch. **Viel Erfolg!**

In [40]:
@t.singledispatch_traverser
def my_traverser(obj: object):

    return obj

@my_traverser.register(str)
def _(s: str):
    return s + s

@my_traverser.register(bool)
def _(b: bool):
    return b

@my_traverser.register(int)
def _(n: int):
    return n * 2

@my_traverser.register(float)
def _(x: float):
    return x * 2

@my_traverser.register(list)
def _(xs: list, *, traverse):
    inner = [traverse(x) for x in xs]
    return inner + inner

@my_traverser.register(tuple)
def _(xs: tuple, *, traverse):
    inner = tuple(traverse(x) for x in xs)
    return inner + inner

@my_traverser.register(set)
def _(xs: set, *, traverse):
    return {traverse(x) for x in xs}

@my_traverser.register(dict)
def _(d: dict, *, traverse):
    return {traverse(k): traverse(v) for k, v in d.items()}

Um Ihren Traverser zu testen, können Sie die folgenden Tests ausführen. Wenn Ihr Traverser korrekt funktioniert, sollten die Tests keinen Fehler werfen.

In [41]:
test_data = [
    ("Wau", "WauWau"),
    (4, 8),
    (-3.5, -7.0),
    (["a", 3], ["aa", 6, "aa", 6]),
    ({"X": 1, 3: ["a"]}, {"XX": 2, 6: ["aa", "aa"]}),
    ((False, False), (False, False, False, False)),
    (True, True),
    (None, None),
]

for data, expected in test_data:
    result = t.traverse(data, my_traverser)
    assert result == expected, f"Der Traverser hat nicht wie erwartet funktioniert: {data=}, {result=}, {expected=}"